In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep

In [4]:
#single family residence from zillow DB
df = prep.prep_df()

In [5]:
df.isnull().sum().sort_values(ascending=False)

regionidcity                  1008
regionidzip                     64
yearbuilt                       27
logerror                         0
assessmentyear                   0
structuretaxvaluedollarcnt       0
regionidcounty                   0
propertycountylandusecode        0
lotsizesquarefeet                0
longitude                        0
latitude                         0
fips                             0
tax_value                        0
sqft                             0
bedrooms                         0
bathrooms                        0
dtype: int64

In [ ]:
#drops columns with < 80% of values filled in
df = prep.impute_values(df)
df = prep.drop_columns(df)

In [12]:
df.dropna(how='any')

,bathrooms,bedrooms,sqft,tax_value,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,assessmentyear,logerror
0,2.0,4.0,3633.0,296425.0,6037.0,34560018.0,-118169806.0,9826.000000,0100,40227.0,3101.0,97329.0,2005.0,222321.0,2016.0,0.042463
1,4.0,3.0,1620.0,847770.0,6037.0,33996200.0,-118438000.0,12857.514983,0100,12447.0,3101.0,96047.0,2011.0,339532.0,2016.0,-0.019546
2,2.0,3.0,2077.0,646760.0,6037.0,34012977.0,-118479243.0,6490.000000,0100,26964.0,3101.0,96152.0,1926.0,210192.0,2016.0,-0.040807
3,1.0,3.0,1244.0,169471.0,6037.0,33953559.0,-118083855.0,6021.000000,0100,13716.0,3101.0,96201.0,1950.0,108040.0,2016.0,0.042412
4,2.0,3.0,1300.0,233266.0,6037.0,33897134.0,-118102953.0,4917.000000,0100,53636.0,3101.0,96193.0,1950.0,77415.0,2016.0,-0.000908
5,2.0,3.0,1222.0,290492.0,6037.0,33889894.0,-118100732.0,5500.000000,0101,53636.0,3101.0,96193.0,1951.0,117839.0,2016.0,-0.316484
6,4.0,4.0,4144.0,1303522.0,6037.0,34143092.0,-117883804.0,17721.957254,0100,11626.0,3101.0,96486.0,2016.0,603000.0,2016.0,0.009215
7,2.5,3.0,1821.0,537949.0,6059.0,33863685.0,-117938506.0,2442.000000,1,24812.0,1286.0,97040.0,2013.0,265668.0,2016.0,0.344151
8,3.5,4.0,2684.0,644990.0,6059.0,33748083.0,-117948108.0,3510.000000,1,34780.0,1286.0,96990.0,2014.0,395167.0,2016.0,-0.133486
9,2.5,3.0,2284.0,934353.0,6059.0,33687181.0,-117753231.0,3398.000000,1,52650.0,1286.0,96954.0,2014.0,478915.0,2016.0,-0.012117


In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

In [ ]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

In [ ]:
(df.structuretaxvaluedollarcnt/df.tax_value).mean()

In [ ]:
df.lotsizesquarefeet.head()

In [ ]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

In [10]:
df_nulls_r = pd.DataFrame(df.apply(lambda x: df.shape[1] -x.count(),axis=1))
df_nulls_r['pct_rows_missing'] = df_nulls_r[0] / df.shape[1]
row_drops = df_nulls_r[df_nulls_r['pct_rows_missing'] >.2]
row_drops['column_names'] = row_drops.index
row_drops = list(row_drops.column_names)
#df_new = df.drop(row_drops,axis=0)

df_nulls_r.max()


0                   2.000
pct_rows_missing    0.125
dtype: float64